# TensorFlow 使用多层感知机(MLP) 识别手写数字

### 1. 下载mnist数据


mnist 是一个手写数字的图片数据库,每一张图片都是0到9中的单个数字。每一张都是抗锯齿(Anti-aliasing)的灰度图,图片大小28x28像素,数字部分被归一化为20*20大小,位于图片的中间位置,保持了原来形状的比例.

tensorflow 提供了一个input_data.py文件，专门用于下载mnist数据，通过下面的代码调用：

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
mnist = read_data_sets("MNIST_data/", one_hot=True)

https://s3.amazonaws.com/lasagne/recipes/datasets/mnist/
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


下载完后会在当前目录下看到 一个名为 'MNIST_data'的文件夹，里面是下载的mnist数据，四个压缩包。压缩包里的内容分别是：

|文件	                                     |内容|
|---------------------------|----|
|train-images-idx3-ubyte.gz	|训练集图片 - 55000 张 训练图片, 5000 张 验证图片|
|train-labels-idx1-ubyte.gz	|训练集图片对应的数字标签|
|t10k-images-idx3-ubyte.gz	|测试集图片 - 10000 张 图片|
|t10k-labels-idx1-ubyte.gz	|测试集图片对应的数字标签|


>因为网络问题无法访问 原来的 [SOURCE_URL](http://yann.lecun.com/exdb/mnist/), 所以根据[这里](https://stackoverflow.com/questions/33731875/tensorflow-ioerror-errno-socket-error-errno-104-connection-reset-by-peer)  的建议修改`anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py` 里的链接为'
https://s3.amazonaws.com/lasagne/recipes/datasets/mnist/'

In [2]:
print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)
print(mnist.validation.images.shape,mnist.validation.labels.shape)
type(mnist)

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)
(5000, 784) (5000, 10)


tensorflow.contrib.learn.python.learn.datasets.base.Datasets

### 2.  实现MLP算法

多层感知机（Multi-Layer Perception， MLP），也叫多层神经网络 或全连接神经网络(FCN, Fully Connected Network)

In [3]:
import tensorflow as tf

In [4]:
in_units = 784
h1_units = 300
output_units = 10

w1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1))
b1 = tf.Variable(tf.zeros([h1_units]))
w2 = tf.Variable(tf.zeros([h1_units, output_units]))
b2 = tf.Variable(tf.zeros([10]))

x = tf.placeholder(tf.float32, [None, in_units])
keep_prob = tf.placeholder(tf.float32)

定义网络结构

In [5]:
hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, w2) + b2)

定义 损失函数 和 优化器

In [6]:
y_ = tf.placeholder(tf.float32, [None, output_units])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)

In [7]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
batch_size=100

for i in range(batch_size*30):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    train_step.run({x:batch_xs, y_: batch_ys, keep_prob:0.75})

In [8]:
correct_prediction= tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x:mnist.test.images, y_: mnist.test.labels, keep_prob:1}))

0.9798
